# Simple example of time domain beamforming

Let's imagine that we have one or more idealised point sound sources with spherical spreading impinging upon an n-element linear microphone array.

The situation is depicted in the figure below.

![Linear array](figures/linear_array.png)

Note that there are two different frames of reference here. 
There is the frame of reference for the location and orientation of the array.
For this frame of reference we use the unit circle, with the centre of the array at
the origin, and the perpendicular to the array (at $0^o$ rotation) pointing along
the x-axis. 
We use θ to denote the angle of the perpendicular of the array relative to
the unit circle.
For the bearing of contacts relative to the array we use a frame of reference
where $0^o$ denotes pointing directly up the array (forward end-fire),
$90^o$ is pointing perpendicular to the array (broadside) and
$180^o$ is directly down the array (aft end-fire).
We use φ to denote the angle of a signal relative to the array bearing.

Note that we are using point sources, not the plane-wave approximation.

We have used the following helpful links in developing the code for this example:

* [Time delay estimation for passive sonar signal processing](https://ieeexplore.ieee.org/document/1163560)
* [Jupyter User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html)
* https://en.wikipedia.org/wiki/Wavelength
* [Equations for Plane Waves, Spherical Waves, and Gaussian Beams](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118939154.app3)
* https://github.com/bokeh/bokeh/blob/1.3.2/examples/howto/layouts/dashboard.py
* [Beamforming in the Time Domain](https://onlinelibrary.wiley.com/doi/10.1002/9781119293132.ch11)


For a linear array, a delay of 0 implies a beam-steer direction of broadside (perpendicular
to the array).

A delay equivalent to the spacing between two adjacent elements divided by the velocity of
the soundwave implies a beam steering direction of either endfire (parallel to the array),
where the endfire (front or back) is determined by the sign of the delay

More generally, a delay of $d_t = \cos\theta \frac{d}{c}$ corresponds to a beam-steer at
an angle $\theta$ for $\theta \in [0^o, 180^o]$.

Given that we are dealing with discrete time signals, the delay cannot be chosen arbitrarily.

Instead, for sample rate $S$, we have

$$ \cos\theta = \frac{c}{d} d_t = \frac{c}{d} \frac{n}{S} $$

where

$$ -\frac{Sd}{c} < n < \frac{Sd}{c} $$


## Import modules

In [ ]:
from ipywidgets import interact, fixed
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label, Span, Legend, LegendItem
from bokeh.layouts import grid, column, gridplot
from bokeh.models import CustomJS, Slider, ColumnDataSource

import seaborn as sns
output_notebook()

## Set up parameters

In order to create interactive Bokeh widgets, it is necessary to separate the plotting and the 
calculating code, so that an update function can be written, which simple updates the data in
the plots.

See:
* https://github.com/bokeh/bokeh/blob/2.3.0/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb

We create a class whose single instance will hold all the information regarding this example.

In [ ]:
class Info(object):
    '''Holder for global information'''
    
    def __repr__(self):
        ''' Print a list of the attributes of an instance of the class'''
        print_str = '['
        for attr, value in self.__dict__.items():
            print_str += f"'{attr}',"
        print_str = print_str[:-1] + ']'
        return print_str
    
    def __str__(self):
        ''' Print a list of the attributes and their values for an
        instance of the class'''
        print_str = ''
        for attr, value in self.__dict__.items():
            print_str += f' {attr} : {value}\n'
        return print_str
    
info = Info()

In [ ]:
def calculate_everything(info,
                         num_elements,
                         array_angle,
                         src_radius,
                         wavelength,
                         t_0,
                         look_index,
                         ):
    '''info contains all the re-useable data associated with this
    example.
    The other values are all values that we would like to vary in the
    plot below'''
#    pass
#if True:
    
    # Array characteristics
    info.spacing = 0.1 # metres
    info.x_location = 0 # metres
    Arr = []
    info.array_length = info.spacing*(num_elements-1)
    for k in range(num_elements):
        x_pos = -(info.array_length/2 - 
                  k*info.spacing)*np.sin(array_angle)
        y_pos = (info.array_length/2 - 
                 k*info.spacing)*np.cos(array_angle)
        Arr.append([x_pos,y_pos])
    #Arr = np.array(Arr)
    info.Arr = Arr
    info.col = sns.color_palette(None, num_elements).as_hex()

    # Source characteristics
    amp_src = 1
    src_angle = np.pi/6
    Src = {}
    Src['position'] = []
    Src['absolute_direction'] = []
    Src['amplitude'] = []
    Src['wavelength'] = []
    for n in range(num_sources):
        x = src_radius*np.cos(n*src_angle)
        y = src_radius*np.sin(n*src_angle)
        Src['position'].append([x,y]) # Source position
        Src['absolute_direction'].append(n*src_angle)
        Src['amplitude'].append(amp_src) # Source amplitude
        Src['wavelength'].append(wavelength) # Source wavelength
    info.Src = Src

    π = np.pi
    c = 340 # m/s (Speed of sound)

    info.sample_rate = 48000

    # The time-domain beamforming does not work well if the sample length
    # is an integer multiple of the period of the source signal.
    info.sample_length = 2.4*Src['wavelength'][0]/c # seconds

    N = 500
    info.x_min = -1
    info.x_max = 1
    x = np.linspace(info.x_min, info.x_max, N)
    y = np.linspace(info.x_min, info.x_max, N)    
    xx, yy = np.meshgrid(x, y)
    
    # We need to get enough time samples, so that we can shift
    # by num_elements * (m[-1] - m[0]) without wrapping around
    
    m = np.arange(int(-np.floor(info.spacing*info.sample_rate/c)),
                  int(np.ceil(info.spacing*info.sample_rate/c)),1)
    info.m = m
    cos_phi = c/info.spacing*m/info.sample_rate
    info.phi = np.arccos(cos_phi)
    
    t = np.arange(t_0, 
                  t_0 + info.sample_length 
                       + num_elements*(m[-1]-m[0])/info.sample_rate,
                       1/info.sample_rate)
    info.t = t
    print(m[0],m[-1],num_elements*(m[-1]-m[0]), len(info.t))
    print(info.sample_length*info.sample_rate)

    Z = 0*xx
    for n in range(num_sources):
        [s_x, s_y] = Src['position'][n] #location of source
        A = Src['amplitude'][n]
        λ = Src['wavelength'][n] # metres
        f = c/λ # Hz

        print(f'Source {n+1}: A = {A}, ',
              f'f = {f} Hz, λ = {λ} m , T = {1/f:0.2e} s') 

        rr = np.sqrt((xx-s_x)*(xx-s_x) + (yy-s_y)*(yy-s_y))

        Z = Z + A/rr*np.sin(2*π/λ*(rr - c*t_0))
    info.Z = Z

    Y = []
    for k in range(num_elements):
        F = 0*info.t
        for n in range(num_sources):
            [s_x, s_y] = Src['position'][n] #location of source
            A = Src['amplitude'][n]
            λ = Src['wavelength'][n] # metres
            f = c/λ # Hz

            r = np.sqrt((Arr[k][0]-s_x)*(Arr[k][0]-s_x) + 
                        (Arr[k][1]-s_y)*(Arr[k][1]-s_y))
            F = F + A/r*np.sin(2*π/λ*(r - c*t))
        Y.append(F)
    info.Y = Y
        
    #Beamforming

    beams = 0*info.phi
    sum_F = {}
    roll = {}
    for j in m:
        sum_F[j] = 0*Y[0]
        for k in range(num_elements):
            sum_F[j] = sum_F [j]+ np.roll(Y[k],j*k)
            if j == look_index:
                pass
        sum_F[j] = sum_F[j]/num_elements
        beams[j] = np.sum(np.abs(sum_F[j]))/len(sum_F[j])
    info.beams = beams

    return info


In [ ]:
def plot_everything(info,
                     num_elements,
                     array_angle,
                     src_radius,
                     wavelength,
                     t_0,
                     look_index,
                     ):
#    pass
#if True:
    
    # Spacial Window
    p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
              )
    p.xaxis.axis_label = p.yaxis.axis_label = 'Distance (m)'
    p.x_range.range_padding = p.y_range.range_padding = 0

    # must give a vector of image data for image parameter
    info.spacial = p.image(image=[info.Z],
            x=info.x_min, y=info.x_min,
            dw=(info.x_max-info.x_min), dh=(info.x_max-info.x_min), 
            palette="Spectral11", level="image")

    info.array_glyphs = []
    for k in range(num_elements):
        info.array_glyphs.append(p.circle(info.Arr[k][0],info.Arr[k][1],
                 size=10,
                 fill_color=info.col[k],
                 legend_label=f'Array Element {k}',
                 ))

    p.grid.grid_line_width = 0.5
    p.legend.click_policy="hide"
    info.p = p

    # Time Series Window
    q = figure()
    q.xaxis.axis_label = 'Time (s)'
    q.yaxis.axis_label = 'Pressure'

    info.time_series = []
    for k in range(num_elements):
        info.time_series.append(q.line(info.t,info.Y[k],
                    color=info.col[k],legend_label=f'Array Element {k}'))

    #info.sum = q.line(info.t,sum_F[look_index],color='black',
    #       legend_label=f'Delay-sum φ:{np.rad2deg(info.phi[look_index]):0.2f}°')

    q.legend.click_policy="hide"
    info.q = q

    # Beamforming Window

    b = figure()
    b.xaxis.axis_label = 'Steering Angle (degrees)'
    b.yaxis.axis_label = 'Pressure'
    info.beampattern_points = b.circle(np.rad2deg(info.phi),
                                info.beams, legend_label='Beampattern')
    info.beampattern_line =  b.line(np.rad2deg(info.phi),info.beams, 
                                    legend_label='Beampattern')
    info.look_line = Span(location=np.rad2deg(info.phi[look_index]),
                     dimension='height',
                     line_color='green', line_width=3,
                    )

    info.source_line = []
    for n in range(num_sources):
        info.source_line.append(Span(location=90+np.rad2deg(array_angle-info.Src['absolute_direction'][n]),
                           dimension='height',
                           line_color='red', line_width=2
                          ))
    b.line([[], []], legend_label='Look direction', line_color="green", line_width=2)
    b.line([[], []], legend_label='Source direction', line_color="red", line_width=2)
    b.renderers.extend([info.look_line, *info.source_line])

    info.b = b
    
    return info

In [ ]:
num_elements = 6
array_angle = np.deg2rad(20) # radians
num_sources = 1
src_radius = 100
wavelength = 0.15
t_0 = 0
look_index = 15

In [ ]:
info = calculate_everything(info,num_elements,array_angle,src_radius,
                           wavelength,t_0,look_index)
#print(info)

In [ ]:
print(array_angle,info.Arr[0][0])

In [ ]:
info = plot_everything(info,num_elements,array_angle,src_radius,
                           wavelength,t_0,look_index)

In [ ]:
info.p

In [ ]:
def update(x,
           aa,
           dd,
           ll,
          ):
    array_angle = np.deg2rad(aa)
    print(info.Arr[0][0])
    x = calculate_everything(x,num_elements,array_angle,dd,
                           wavelength,t_0,ll)
    
    x.look_line.location=np.rad2deg(x.phi[ll])
    x.spacial.data_source.data['image'] = [x.Z]
    

    print(x.Arr[0][0])
    #array_angle
    for k in range(num_elements):
        x.array_glyphs[k].data_source.data['x'] = [x.Arr[k][0]]
        x.array_glyphs[k].data_source.data['y'] = [x.Arr[k][1]]
        x.time_series[k].data_source.data['y'] = x.Y[k]
        #x.array_glyphs[k].size = x.array_glyphs[k].size + 5
        #info.array_glyphs[k].
    push_notebook()
    print(aa,array_angle)

In [ ]:
show(gridplot([info.p, info.q, info.b], ncols=3), notebook_handle=True)
interact(update, x=fixed(info), 
         aa=(-90, 90, 0.1), 
         dd=(1,100), 
         ll = (info.m[0],info.m[-1],1),
         #lambda = (0.05,
        )
print(array_angle)

In [ ]:
ww = info.spacial.data_source.data['image'] = [np.random.rand(500,500)]
push_notebook()

In [ ]:
print(info.Arr)
for k in range(num_elements):
    info.array_glyphs[k]

print(info.array_glyphs[0].data_source.data)
push_notebook()

In [ ]:
info.spacial.data_source.data['image'] = [0*info.Z]
push_notebook()

In [ ]:
k=5
info.time_series[k].data_source.data['y'] = 0*info.Y[k]
push_notebook()

In [ ]:
import numpy as np

from bokeh.layouts import grid, column
from bokeh.models import CustomJS, Slider, ColumnDataSource
#from bokeh.plotting import figure, output_file, show

tools = 'pan'


def bollinger():
    # Define Bollinger Bands.
    upperband = np.random.randint(100, 150+1, size=100)
    lowerband = upperband - 100
    x_data = np.arange(1, 101)

    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = figure(x_axis_type='datetime', tools=tools)
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    p.plot_height = 600
    p.plot_width = 800
    p.grid.grid_line_alpha = 0.4
    return [p]


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

    callback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, offset=offset_slider),
                        code="""
        const data = source.data;
        const A = amp.value;
        const k = freq.value;
        const phi = phase.value;
        const B = offset.value;
        const x = data['x']
        const y = data['y']
        for (var i = 0; i < x.length; i++) {
            y[i] = B + A*Math.sin(k*x[i]+phi);
        }
        source.change.emit();
    """)

    amp_slider.js_on_change('value', callback)
    freq_slider.js_on_change('value', callback)
    phase_slider.js_on_change('value', callback)
    offset_slider.js_on_change('value', callback)

    widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = figure(tools=tools)
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
    s2 = figure(tools=tools, x_range=s1.x_range, y_range=s1.y_range)
    s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = figure(tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    return [s1, s2, s3]

l = grid([
    bollinger(),
    slider(),
    linked_panning(),
],)# sizing_mode='stretch_both')

show(l)